In [ ]:
from osgeo import gdal
path = "your path"
hdf_file = path
hdataset = gdal.Open(hdf_file)
subdatasets = hdataset.GetSubDatasets()

for i, sds in enumerate(subdatasets):
    print(f"{i}: {sds[0]}")

In [ ]:
import os
import glob
import numpy as np
from osgeo import gdal
import rasterio
from rasterio.merge import merge
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.crs import CRS
from affine import Affine
from datetime import datetime
from contextlib import ExitStack
from collections import defaultdict

# set path
input_path = 'your path'
output_path = 'your path'
input_folder = input_path
output_folder = output_path

# QA checking
def valid_QA(qa_value):
    return qa_value in [0, 1]

def apply_QA_mask(vi_array, qa_array):
    mask = np.vectorize(valid_QA)(qa_array)
    vi_array[~mask] = np.nan
    return vi_array

def process_modis_file(hdf_path):
    hdf = gdal.Open(hdf_path)
    subdatasets = hdf.GetSubDatasets()

    vi_sds = [sds[0] for sds in subdatasets if '500m 16 days EVI' in sds[0]][0]
    qa_sds = [sds[0] for sds in subdatasets if '500m 16 days pixel reliability' in sds[0]][0]

    vi_ds = gdal.Open(vi_sds)
    qa_ds = gdal.Open(qa_sds)

    vi_array = vi_ds.ReadAsArray().astype(np.float32)
    qa_array = qa_ds.ReadAsArray()

    vi_array[vi_array == -3000] = np.nan
    vi_array = apply_QA_mask(vi_array, qa_array)

    transform = Affine.from_gdal(*vi_ds.GetGeoTransform())
    projection = vi_ds.GetProjection()

    return vi_array, transform, projection

def write_geotiff(output_path, data, transform, projection):
    height, width = data.shape
    with rasterio.open(
        output_path,
        'w',
        driver='GTiff',
        height=height,
        width=width,
        count=1,
        dtype='float32',
        crs=CRS.from_wkt(projection),
        transform=transform,
        nodata=np.nan
    ) as dst:
        dst.write(data, 1)

def reproject_to_wgs84(input_tif, output_tif, dst_resolution=0.0041666667):
    dst_crs = 'EPSG:4326'
    with rasterio.open(input_tif) as src:
        transform, width, height = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, *src.bounds, resolution=dst_resolution)

        kwargs = src.meta.copy()
        kwargs.update({
            'crs': dst_crs,
            'transform': transform,
            'width': width,
            'height': height
        })

        with rasterio.open(output_tif, 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.nearest)

def process_by_date():
    hdf_files = sorted(glob.glob(os.path.join(input_folder, '*.hdf')))
    date_groups = defaultdict(list)

    for f in hdf_files:
        date_str = f.split('.')[1]  
        date_groups[date_str].append(f)

    for date_str, files in date_groups.items():
        arrays = []
        transforms = []
        projections = []
        temp_tifs = []

        # Handle each tile
        for i, f in enumerate(files):
            vi_array, transform, projection = process_modis_file(f)
            temp_tif = os.path.join(output_folder, f"{date_str}_tile{i}.tif")
            write_geotiff(temp_tif, vi_array, transform, projection)
            temp_tifs.append(temp_tif)

        # Concatenate and write to the merged file
        with ExitStack() as stack:
            src_files_to_mosaic = [stack.enter_context(rasterio.open(tif)) for tif in temp_tifs]
            mosaic, out_trans = merge(src_files_to_mosaic)

            merged_path = os.path.join(output_folder, f"{date_str}_merged.tif")
            out_meta = src_files_to_mosaic[0].meta.copy()
            out_meta.update({
                "height": mosaic.shape[1],
                "width": mosaic.shape[2],
                "transform": out_trans,
                "nodata": np.nan
            })

            with rasterio.open(merged_path, "w", **out_meta) as dest:
                dest.write(mosaic)

        # WGS84
        reprojected_path = os.path.join(output_folder, f"{date_str}_WGS84.tif")
        reproject_to_wgs84(merged_path, reprojected_path)

        # delete tif files
        for tif in temp_tifs:
            try:
                os.remove(tif)
            except Exception as e:
                print(f"cannot delete {tif}: {e}")

        try:
            os.remove(merged_path)
        except Exception as e:
            print(f"cannot delete {merged_path}: {e}")

        print(f"complete: {date_str}")

# main
if __name__ == "__main__":
    os.makedirs(output_folder, exist_ok=True)
    process_by_date()